In [1]:
import numpy as np
import scipy
from scipy.stats import dirichlet, multinomial
import pandas as pd
from matplotlib import pyplot as plt


In [2]:
#!jt -t gruvboxd
#!jt -t gruvboxd -T -N

In [3]:
df_data = pd.read_csv('uci-news-aggregator.csv')
df_data.head()

,ID,TITLE,URL,PUBLISHER,CATEGORY,STORY,HOSTNAME,TIMESTAMP
0,1,"Fed official says weak data caused by weather,...",http://www.latimes.com/business/money/la-fi-mo...,Los Angeles Times,b,ddUyU0VZz0BRneMioxUPQVP6sIxvM,www.latimes.com,1394470370698
1,2,Fed's Charles Plosser sees high bar for change...,http://www.livemint.com/Politics/H2EvwJSK2VE6O...,Livemint,b,ddUyU0VZz0BRneMioxUPQVP6sIxvM,www.livemint.com,1394470371207
2,3,US open: Stocks fall after Fed official hints ...,http://www.ifamagazine.com/news/us-open-stocks...,IFA Magazine,b,ddUyU0VZz0BRneMioxUPQVP6sIxvM,www.ifamagazine.com,1394470371550
3,4,"Fed risks falling 'behind the curve', Charles ...",http://www.ifamagazine.com/news/fed-risks-fall...,IFA Magazine,b,ddUyU0VZz0BRneMioxUPQVP6sIxvM,www.ifamagazine.com,1394470371793
4,5,Fed's Plosser: Nasty Weather Has Curbed Job Gr...,http://www.moneynews.com/Economy/federal-reser...,Moneynews,b,ddUyU0VZz0BRneMioxUPQVP6sIxvM,www.moneynews.com,1394470372027


In [4]:
raw_documents = df_data['TITLE'].astype(str).apply(lambda x: x.lower()).array
raw_documents.shape, raw_documents[0:10]

((422419,), <PandasArray>
 [    'fed official says weak data caused by weather, should not slow taper',
        "fed's charles plosser sees high bar for change in pace of tapering",
     'us open: stocks fall after fed official hints at accelerated tapering',
                "fed risks falling 'behind the curve', charles plosser says",
                        "fed's plosser: nasty weather has curbed job growth",
                         'plosser: fed may have to accelerate tapering pace',
                                 "fed's plosser: taper pace may be too slow",
  "fed's plosser expects us unemployment to fall to 6.2% by the end of 2014",
    'us jobs growth last month hit by weather:fed president charles plosser',
              'ecb unlikely to end sterilisation of smp purchases - traders']
 Length: 10, dtype: object)

In [5]:
#Get first 1000 documents
raw_documents = raw_documents[:1000]
raw_documents

<PandasArray>
[                       'fed official says weak data caused by weather, should not slow taper',
                          "fed's charles plosser sees high bar for change in pace of tapering",
                       'us open: stocks fall after fed official hints at accelerated tapering',
                                  "fed risks falling 'behind the curve', charles plosser says",
                                          "fed's plosser: nasty weather has curbed job growth",
                                           'plosser: fed may have to accelerate tapering pace',
                                                   "fed's plosser: taper pace may be too slow",
                    "fed's plosser expects us unemployment to fall to 6.2% by the end of 2014",
                      'us jobs growth last month hit by weather:fed president charles plosser',
                                'ecb unlikely to end sterilisation of smp purchases - traders',
 ...
                     

In [6]:
docs = [d.split() for d in raw_documents]
vocab = list(set(' '.join(raw_documents).split()))
vocab[:10],len(vocab)

(['ever',
  'tumble',
  'chrysler',
  'sales',
  'cathode',
  'video:',
  'death',
  'test',
  'last',
  'walt'],
 2205)

In [7]:
#create word ids
mapped_docs = []
for doc in docs:
    new_doc = []
    vectorized_doc = doc 
    for i in range(len(doc)):
        vectorized_doc[i] = vocab.index(doc[i])
    mapped_docs.append(vectorized_doc)
len(mapped_docs), mapped_docs[:5]

(1000,
 [[981, 720, 1668, 1992, 864, 2040, 1727, 828, 1932, 2204, 1549, 85],
  [248, 1920, 974, 1924, 1968, 1490, 372, 1321, 511, 256, 40, 21],
  [33, 2056, 508, 811, 16, 981, 720, 1874, 541, 1435, 21],
  [981, 2195, 2024, 625, 2184, 2039, 1920, 974, 1668],
  [248, 1145, 93, 703, 694, 480, 2155, 357]])

In [8]:
#True in [(2204 in d) for d in mapped_docs]

In [9]:
len(vocab), len(mapped_docs)

(2205, 1000)

In [10]:
#Number of topics
K = 5

#topic-word matrix
tw_matrix = np.zeros((K,len(vocab)))

#topic assignment list
ta_list = [np.zeros((1,len(d)))[0] for d in docs]



#document-topic matrix
dt_matrix = np.zeros((len(docs),K))


In [11]:
#Randomly intitialize

for d in range(len(docs)):
    for w in range(len(mapped_docs[d])):
        ta_list[d][w] = np.random.randint(0,K)
        
        ti = int(ta_list[d][w])
        wi = int(mapped_docs[d][w])
        tw_matrix[ti, wi] += 1
    
    for t in range(K):
        #Number of words in document d with topic assignment t
        dt_matrix[d, t] = np.where(ta_list[d] == t)[0].shape[0] 

In [12]:
pd.DataFrame(tw_matrix)

,0,1,2,3,4,5,6,7,8,9,...,2195,2196,2197,2198,2199,2200,2201,2202,2203,2204
0,0.0,1.0,0.0,2.0,1.0,1.0,0.0,0.0,2.0,0.0,...,3.0,0.0,1.0,0.0,0.0,0.0,1.0,4.0,0.0,0.0
1,1.0,0.0,0.0,7.0,0.0,0.0,0.0,2.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,3.0
2,0.0,1.0,0.0,4.0,0.0,0.0,0.0,0.0,2.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,3.0
3,1.0,1.0,0.0,11.0,0.0,0.0,2.0,2.0,1.0,0.0,...,0.0,3.0,0.0,0.0,0.0,1.0,0.0,3.0,0.0,3.0
4,0.0,1.0,1.0,4.0,0.0,0.0,0.0,1.0,3.0,0.0,...,3.0,1.0,1.0,0.0,1.0,0.0,0.0,2.0,1.0,7.0


In [13]:
pd.DataFrame(dt_matrix)

,0,1,2,3,4
0,4.0,4.0,2.0,1.0,1.0
1,6.0,2.0,2.0,1.0,1.0
2,2.0,2.0,2.0,2.0,3.0
3,3.0,1.0,1.0,3.0,1.0
4,1.0,4.0,0.0,1.0,2.0
5,3.0,2.0,0.0,0.0,3.0
6,1.0,1.0,2.0,1.0,3.0
7,1.0,5.0,2.0,2.0,4.0
8,1.0,1.0,4.0,2.0,3.0
9,2.0,2.0,3.0,2.0,1.0


In [14]:
#Model paramters
alpha = 3
eta = 1
num_iterations = 20


In [15]:
#calculating P(z_i|*)

#for every word in every document
for iteration in range(num_iterations):
    if iteration % 2 == 0 or iteration == num_iterations-1:
        print(f'{iteration}/{num_iterations}')
    for d_i in range(len(mapped_docs)):
        for w_i in range(len(mapped_docs[d_i])):
            
            init_topic = int(ta_list[d_i][w_i])
            word_id = mapped_docs[d_i][w_i] 


            #z_-i term,
            dt_matrix[d_i, init_topic] -= 1
            tw_matrix[init_topic, word_id] -= 1
            
            if dt_matrix[d_i, init_topic] < 0 or tw_matrix[init_topic, word_id] < 0:
                print(f'error(dt,tw): {dt_matrix[d_i, init_topic]}, {tw_matrix[init_topic, word_id]}')
                

            #word topic means
            probs = np.zeros(K)
            for k in range(K):
                phi_k = (tw_matrix[k, word_id] + eta) / (tw_matrix[k,:].sum() + len(vocab)*eta)
                theta_d = (dt_matrix[d_i,k] + alpha) / (dt_matrix[d_i,:].sum() + K*alpha )
                probs[k] = phi_k*theta_d
                
            # P(z_i|.) = phi_k * theta_d 
            #probs = wt_means*dt_means
            #if d_i == 0:
            #    print(f'''(d {d_i}, w {w_i}): word-topic:{wt_means} 
            #          doc-topic{dt_means} 
            #          word-prob{probs}
            #          ------------()    
            #Normalize, necessary due to rounding errors
            probs = probs/probs.sum()
            
            #Multinomial draws
            new_topic = np.argmax(np.random.multinomial(1,probs))
            dt_matrix[d_i, new_topic] += 1
            tw_matrix[new_topic, w_i] += 1
            if dt_matrix[d_i, new_topic] < 0 or tw_matrix[new_topic, word_id] < 0:
                print(f'new topic(dt,tw): { dt_matrix[d_i, new_topic]}, {tw_matrix[new_topic, word_id]}')
            ta_list[d_i][w_i] = new_topic
            




0/20
error(dt,tw): 5.0, -1.0
error(dt,tw): 4.0, -1.0
error(dt,tw): 2.0, -1.0
error(dt,tw): 1.0, -1.0
error(dt,tw): 4.0, -1.0
error(dt,tw): 3.0, -1.0
error(dt,tw): 2.0, -1.0
error(dt,tw): 5.0, -1.0
error(dt,tw): 2.0, -1.0
error(dt,tw): 1.0, -1.0
error(dt,tw): 5.0, -1.0
error(dt,tw): 4.0, -1.0
error(dt,tw): 2.0, -1.0
error(dt,tw): 0.0, -1.0
error(dt,tw): 1.0, -1.0
error(dt,tw): 4.0, -1.0
error(dt,tw): 3.0, -1.0
error(dt,tw): 1.0, -1.0
error(dt,tw): 2.0, -1.0
error(dt,tw): 5.0, -1.0
error(dt,tw): 2.0, -1.0
error(dt,tw): 1.0, -1.0
error(dt,tw): 1.0, -1.0
error(dt,tw): 0.0, -1.0
error(dt,tw): 1.0, -1.0
error(dt,tw): 2.0, -1.0
error(dt,tw): 1.0, -1.0
error(dt,tw): 0.0, -1.0
error(dt,tw): 0.0, -1.0
error(dt,tw): 5.0, -1.0
error(dt,tw): 3.0, -1.0
error(dt,tw): 2.0, -1.0
error(dt,tw): 6.0, -1.0
error(dt,tw): 5.0, -1.0
error(dt,tw): 4.0, -1.0
error(dt,tw): 0.0, -1.0
error(dt,tw): 4.0, -1.0
error(dt,tw): 2.0, -1.0
error(dt,tw): 0.0, -1.0
error(dt,tw): 1.0, -1.0
error(dt,tw): 5.0, -1.0
error(dt,tw

ValueError: pvals < 0, pvals > 1 or pvals contains NaNs

In [ ]:
# Gibbs sampling loop with assignment history
plt.plot()

In [18]:
df_tw = pd.DataFrame(tw_matrix)
df_tw.head(), len(vocab)

(    0      1      2      3      4      5      6      7      8     9     ...  \
 0  184.0  191.0  203.0  203.0  187.0  189.0  163.0  147.0  105.0  76.0  ...   
 1  194.0  210.0  204.0  200.0  213.0  175.0  162.0  118.0  116.0  66.0  ...   
 2  217.0  213.0  206.0  189.0  189.0  193.0  163.0  125.0  101.0  87.0  ...   
 3  213.0  195.0  188.0  203.0  211.0  168.0  167.0  137.0   90.0  71.0  ...   
 4  198.0  196.0  201.0  191.0  168.0  194.0  149.0  150.0   89.0  67.0  ...   
 
    2195  2196  2197  2198  2199  2200  2201  2202  2203  2204  
 0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  
 1   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  
 2   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  -1.0  
 3   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  
 4  -1.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  
 
 [5 rows x 2205 columns], 2205)

In [ ]:
word_lists = []
for k in range(K):
    topic_k_words = df_tw.iloc[k, :].array
    #Get top 10 words for topic k
    top_words_ind = np.argpartition(topic_k_words, -10)[-10:]
    top_words = [vocab[v] for v in top_words_ind]
    word_lists.append(top_words)

In [ ]:
df_tw.iloc[0, :], df_tw.iloc[1, :]

In [ ]:
word_lists

In [22]:
tw_matrix[0,:]

array([184., 191., 203., ...,   0.,   0.,   0.])

In [24]:
[] * 5

[]